## MMAI 844 Facial Recognition Demo - Colab Version

<div style="float: left; width: 75%; height: 200px; padding-bottom:400px">
    <img src="https://www.digitalvidya.com/wp-content/uploads/2018/09/Face-Recognition-Python-1280x720.jpg" alt="CNN">
</div>
</div>

In [ ]:
!git clone https://github.com/BenchengW/Facial-Recognition-MMAI844-Tutorial

In [ ]:
! pip install keras-vggface
! pip install keras_preprocessing 
! pip install keras_applications

### Facial Image Classification with Tensorflow pre-trained model

Face recognition is the general task of identifying and verifying people from photographs of their face.

### Step 1.1: Imports Libraries
Import libraries and define environment variables

In [ ]:
import cv2
import numpy as np
import os
import math
from matplotlib import pyplot as plt
%matplotlib inline
import cv2
print(cv2.__version__)
%matplotlib inline
cv2.startWindowThread()
from os import listdir
from PIL import Image
import warnings
warnings.filterwarnings(action='once')
import urllib.request

In [ ]:
import keras
import keras_vggface
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from tensorflow.keras import datasets, layers, models
from keras.optimizers import RMSprop, SGD
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
from numpy import asarray

<div style="float: left; width: 75%; height: 200px; padding-bottom:350px">
    <img src="https://lh3.googleusercontent.com/IdcOyMJ4hCDvSJXWBo1Rxr1BTM9fQWoxShs0tdS93bpyQ1K6vIog_mV9LrfE0DwKK61X2fHY51AAbPJTkOOMDUVxaiE32JsGog74k3lnXKXPefpd_fSC3divPG3AEEQhaith6S47" alt="CNN">
</div>
</div>

### Step 1.2: Load Data/Image
#### Read and Write Images
``` python
cv2.imwrite(file_path (str), image (numpy.ndarray))
cv2.imread(file_path (str), read_mode (int))```
#### Read Modes
-  ```1 = cv2.IMREAD_COLOR```
-  ```0 = cv2.IMREAD_GRAYSCALE```
- ```-1 = cv2.IMREAD_UNCHANGED```

Load a Sample Simith image using opencv library

In [ ]:
img =cv2.imread("/content/Facial-Recognition-MMAI844-Tutorial/sample/Smith.jpg",1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

You can also Take a Picture or start video with your webcam

In [ ]:
def plt_show(image, title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image, cmap="Greys_r")
    plt.show()
    
########################################
#Take a Picture when you camera is free
#########################################
webcam = cv2.VideoCapture(0)
ret, frame = webcam.read()
print(ret)
webcam.release()
if ret:
    plt_show(frame)
else:
    print("Camera is not available")

### Step 1.3: Detecting Faces with OpenCV and front face detector xml 
``` python
detector = cv2.CascadeClassifier( xml_file_path)
face_coord = detector.detectMultiScale(image, scale_factor, min_neighbors, min_size, flags)
```
face_coord: Numpy array with rows equal to [x, y, width, height]

Let import a image to detect faces

In [ ]:
frame= cv2.imread('/content/Facial-Recognition-MMAI844-Tutorial/sample/channing_tatum.jpg')
name= "channing_tatum"
plt_show(frame)

In [ ]:
detector = cv2.CascadeClassifier("/content/Facial-Recognition-MMAI844-Tutorial/xml/frontal_face.xml")

scale_factor = 1.2
min_neighbors = 5
min_size = (40, 40)
biggest_only = True
flags = cv2.CASCADE_FIND_BIGGEST_OBJECT | \
            cv2.CASCADE_DO_ROUGH_SEARCH if biggest_only else \
            cv2.CASCADE_SCALE_IMAGE
        
faces_coord = detector.detectMultiScale(frame,
                                        scaleFactor=scale_factor,
                                        minNeighbors=min_neighbors,
                                        minSize=min_size,
                                        flags=flags)
print("Type: " + str(type(faces_coord)))
print("this is face coordinator in the picture is {}".format(faces_coord))
print("Face is successfully detected!! Let draw a box on the picture")

In [ ]:
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        w_rm = int(0.2 * w / 2) 
        cv2.rectangle(image, (x + w_rm, y), (x + w - w_rm, y + h), 
                              (0, 0, 255), 8)

In [ ]:
draw_rectangle(frame,faces_coord)
cv2.putText(frame, name,    
                            (faces_coord[0][0], faces_coord[0][1]),
                            cv2.FONT_HERSHEY_SIMPLEX , 2, (66, 53, 243), 6)
plt_show(frame)

In [ ]:
#Wrap up the code into a face detector module
class FaceDetector(object):
    def __init__(self, xml_path):
        self.classifier = cv2.CascadeClassifier(xml_path)
    
    def detect(self, image, biggest_only=True):
        scale_factor = 1.2
        min_neighbors = 5
        min_size = (30, 30)
        biggest_only = True
        flags = cv2.CASCADE_FIND_BIGGEST_OBJECT | \
                    cv2.CASCADE_DO_ROUGH_SEARCH if biggest_only else \
                    cv2.CASCADE_SCALE_IMAGE
        faces_coord = self.classifier.detectMultiScale(image,
                                                       scaleFactor=scale_factor,
                                                       minNeighbors=min_neighbors,
                                                       minSize=min_size,
                                                       flags=flags)
        return faces_coord

### Step 1.4: Cut Faces and resize faces

for (x, y, w, h) in faces_coord:
    cv2.rectangle(frame, (x, y), (x + w, y + h), (150, 150, 0), 8)
plt_show(frame) 

In [ ]:
def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        w_rm = int(0.2 * w / 2)
        faces.append(image[y: y + h, x + w_rm: x + w - w_rm])
         
    return faces

In [ ]:
Cut_Face = cut_faces(frame, faces_coord)

In [ ]:
plt_show(Cut_Face[0])

In [ ]:
def resize(images, size=(224, 224)):
    images_norm = []
    for image in images:
        if image.shape < size:
            image_norm = cv2.resize(image, size, 
                                    interpolation = cv2.INTER_AREA)
        else:
            image_norm = cv2.resize(image, size, 
                                    interpolation = cv2.INTER_CUBIC)
        images_norm.append(image_norm)

    return images_norm 

In [ ]:
resize_faces = resize(Cut_Face)

In [ ]:
plt_show(resize_faces[0])

### Step 1.5: Standerdize the image

In [ ]:
Resize_face = resize_faces[0]

In [ ]:
plt.imshow(Resize_face/225.0)

## Step2:Pre-trained Model in Tensorflow

### Step 2.1 Install libraries

VGGFace and VGGFace2 Models

The VGGFace refers to a series of models developed for face recognition and demonstrated on benchmark computer vision datasets by members of the Visual Geometry Group (VGG) at the University of Oxford.

<div style="float: left; width: 75%; height: 200px; padding-bottom:280px">
    <img src="https://pbs.twimg.com/media/Dn2GdPfW0AE0nf7?format=jpg&name=4096x4096" alt="CNN">
</div>
</div>

In [ ]:
!pip show keras-vggface

### Step 2.1: Verify Data and Predict the image using Pre-train model

In [ ]:
# from numpy import asarray
# # extract a single face from a given photograph
def extract_face_from_file(filename, required_size=(224, 224)):
    # load image from file
    pixels = plt.imread(filename)
    detector = FaceDetector("/content/Facial-Recognition-MMAI844-Tutorial/xml/frontal_face.xml")
    # create the detector, using default weights
    faces_coord = detector.detect(image=pixels)
    faces = cut_faces(pixels, faces_coord)
    faces = resize(faces)
    return faces[0]

In [ ]:
Sample= plt.imread('/content/Facial-Recognition-MMAI844-Tutorial/sharon_stone1.jpg')
plt.imshow(Sample)

In [ ]:
# load the photo and extract the face
extract_face = extract_face_from_file('/content/Facial-Recognition-MMAI844-Tutorial/sample/sharon_stone1.jpg')
# plot the extracted face
plt.imshow(extract_face)

### Step 2.2 Check Model Input and output, and model summary

In [ ]:
from keras_vggface.vggface import VGGFace
# create a vggface2 model
model = VGGFace(model='resnet50')
# summarize input and output shape
print('Inputs: %s' % model.inputs)
print('Outputs: %s' % model.outputs)

We can see that the model expects input color images of faces with the shape of 244×244 and the output will be a class prediction of 8,631 people. the input dimension is 4. This means that you have to reshape your training set with .reshape(n_images, 286, 384, 1)

In [ ]:
#prints out summary of model
model.summary()

### Step 2.2 Prepare input for faces

In [ ]:
Face_array = asarray(extract_face,'float32')
Preprocess_face = preprocess_input(Face_array, version=2)
print(Preprocess_face.shape)
Preprocess_face_input = Preprocess_face.reshape(1, 224, 224, 3)
print(Preprocess_face_input.shape)

In [ ]:
# perform prediction
yhat = model.predict(Preprocess_face_input)

In [ ]:
# convert prediction into names
results = decode_predictions(yhat)
# display most likely results
for result in results[0]:
    print('%s: %.3f%%' % (result[0], result[1]*100))

In [ ]:
def draw_rectangle_with_label(image, label):
    faces_coord = detector.detectMultiScale(image,
                                        scaleFactor=scale_factor,
                                        minNeighbors=min_neighbors,
                                        minSize=min_size,
                                        flags=flags)
    draw_rectangle(image,faces_coord)
    cv2.putText(image, name,    
                            (faces_coord[0][0], faces_coord[0][1]),
                            cv2.FONT_HERSHEY_SIMPLEX , 1, (66, 53, 243), 3)

    plt.imshow(image)
    plt.show()
    for result in results[0]:
        print('%s: %.3f%%' % (result[0], result[1]*100))

draw_rectangle_with_label(Sample, results)

### Step 3 Use Pre-trained Model for Your Photo

#### We are going to use a public dataset for training our own model. 
You can download the dataset from here: https://www.kaggle.com/dansbecker/5-celebrity-faces-dataset

This is a small dataset for experimenting with computer vision techniques. It has a training directory containing 14-20 photos each of the celebrities

--Ben Afflek

--Elton John

--Jerry Seinfeld

--Madonna

--Mindy Kaling
    
The validation directory has 5 photos of each celebrity.

The photos haven't been cropped for consistent aspect ratios. With so few training photos, this an especially interesting test of computer vision techniques.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
pixels = plt.imread("/content/Facial-Recognition-MMAI844-Tutorial/train/ben_afflek/httpwwwhillsindcomstorebenjpg.jpg")
plt.imshow(pixels)

### Step 3.1 Prepare input data and parameters

In keras there is a image preprocessing function that can not only do the image preprocessing but also run data augmentation for you.
For more information please refer to: https://keras.io/api/preprocessing/image/

In [ ]:
img_height=224
img_width=224
batch_size=16

### Step 3.2 Specify the data folder train and val

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image 
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    #shear_range=0.2,
    vertical_flip=False,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    '/content/Facial-Recognition-MMAI844-Tutorial/train',
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/Facial-Recognition-MMAI844-Tutorial/val',
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='categorical')

In [ ]:
#custom parameters
nb_class = 5
hidden_dim = 200
nb_train_samples = 93

nb_validation_samples = 25
epochs = 10
batch_size = 16
numclasses = 5

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

In [ ]:
custom_vgg_model.summary()

In [ ]:
lr = 1e-5
decay = 1e-7 #0.0
optimizer = RMSprop(lr=lr, decay=decay)
custom_vgg_model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Step 3.3 Train model

In [ ]:
history = custom_vgg_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

### Step 3.4 Visualize the trainning and validation accuracy and loss

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
training_acc = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

fig=plt.figure(figsize=(12, 4))
# Visualize loss history
fig.add_subplot(121)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Training Loss', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Training Loss/Acc')

# Get training and test loss histories
val_acc = history.history['val_accuracy']
training_acc = history.history['accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(val_acc) + 1)

# Visualize loss history
fig.add_subplot(122)
plt.plot(epoch_count, val_acc, 'r--')
plt.plot(epoch_count, training_acc, 'b-')
plt.legend(['Validation Accuracy', 'Training Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.show();

### Step 3.5 Save the model and run prediction

In [ ]:
saveweight =  'celebriytag_weight.h5'
model.save_weights(saveweight)

In [ ]:
labels = ['ben_afflek',  'elton_john',  'jerry_seinfeld',  'madonna',  'mindy_kaling']
test_imgs = ['/content/Facial-Recognition-MMAI844-Tutorial/val/ben_afflek/123MTENDgMDUODczNDcNTcjpg.jpg']


test_img = '/content/Facial-Recognition-MMAI844-Tutorial/val/ben_afflek/123MTENDgMDUODczNDcNTcjpg.jpg'
img = image.load_img(test_img, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x /= 255.
classes = custom_vgg_model.predict(x)
result = np.squeeze(classes)
result_indices = np.argmax(result)
    
img = cv2.imread(test_img, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.axis('off')
plt.title("{}, {:.2f}%".format(labels[result_indices], result[result_indices]*100))
plt.imshow(img)